In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

/Users/VoThinhPhat/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5_vi_en_translation")
tokenizer = T5Tokenizer.from_pretrained("t5_vi_en_translation")

# Move model to the selected device (GPU or CPU)
model.to(device)

Using device: cpu


In [ ]:
def translate_text(input_text, model, tokenizer, device):
    # Preprocess the input text
    input_ids = tokenizer.encode(f"translate Vietnamese to English: {input_text}", return_tensors="pt").to(device)
    
    # Generate the translation (decoding the output token IDs)
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        translation_ids = model.generate(input_ids, max_length=256, num_beams=5, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
    
    # Decode the generated tokens into the output text (English translation)
    output_text = tokenizer.decode(translation_ids[0], skip_special_tokens=True)
    
    return output_text

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo

In [ ]:
# Test the translation on a new Vietnamese sentence
src_sentence = "Theo lãnh đạo Sở Y tế, 3 người này không có triệu chứng sốt, ho, khó thở, đã được lấy mẫu xét nghiệm và cách ly tập trung."
translated_sentence = translate_text(src_sentence, model, tokenizer, device)

# Print the result
print("Original Vietnamese Sentence: ", src_sentence)
print("Translated English Sentence: ", translated_sentence)